In [45]:
%ls data

FONNESBECK_ADT_20151202.csv        FONNESBECK_ICD9_20151202.csv*
FONNESBECK_BMI_20151202.csv*       FONNESBECK_LAB_20151202.csv*
FONNESBECK_BP_20151202.csv*        FONNESBECK_MED_20151202.csv*
FONNESBECK_CPT_20151202.csv*       FONNESBECK_phenotype_20151202.csv*
FONNESBECK_EGFR_20151202.csv*      Fonnesbeck_DD_2014102014.xlsx


In [81]:
import pandas as pd
import datetime

adt = pd.read_table('data/FONNESBECK_ADT_20151202.csv', encoding='latin1', sep = ',', infer_datetime_format=True, parse_dates=['Admission_date','Event_Date','DISCHARGE_DATE'])
pheno = pd.read_table('data/FONNESBECK_phenotype_20151202.csv', encoding='latin1', sep = ',', infer_datetime_format=True, parse_dates=['DOB','DOD'])
cpt = pd.read_table('data/FONNESBECK_CPT_20151202.csv', encoding='latin1', sep = ',', infer_datetime_format=True, parse_dates=['Event_date'])

In [139]:
svc = pd.read_excel('data/FONNESBECK_DD_2014102014.xlsx',sheet_name='Service code', sep = ',')
svc.rename(columns = {"Service Code":"SVC", "Service Code Desc":"Desc"}, inplace = True)

In [182]:
adt.Event = pd.Categorical(adt.Event,categories = ['Admit','Transfer','Discharge'])
adt = adt.sort_values(by = ['RUID','Admission_date','Event','Event_Date']).reset_index(drop = True)

In [183]:
adt.head()

,RUID,Event,Admission_date,Event_Date,SRV_CODE,CHIEF_COMPLAINT,DISCHARGE_DATE
0,50135262,Admit,2007-02-08,2007-02-08,ORT,R FEMUR FX,2007-02-12
1,50135262,Transfer,2007-02-08,2007-02-08,ORT,R FEMUR FX,2007-02-12
2,50135262,Transfer,2007-02-08,2007-02-09,ORT,R FEMUR FX,2007-02-12
3,50135262,Discharge,2007-02-08,2007-02-12,ORT,R FEMUR FX,2007-02-12
4,50135262,Admit,2007-08-03,2007-08-03,CAR,CP,2007-08-06


In [82]:
adt.isnull().sum()/adt.shape[0]

RUID               0.000000
Event              0.000000
Admission_date     0.012845
Event_Date         0.000000
SRV_CODE           0.000000
CHIEF_COMPLAINT    0.007628
DISCHARGE_DATE     0.016934
dtype: float64

In [97]:
adt.shape

(121530, 7)

In [102]:
adt.describe(include='all')

,RUID,Event,Admission_date,Event_Date,SRV_CODE,CHIEF_COMPLAINT,DISCHARGE_DATE
count,1.215300e+05,121530,119969,121530,121530,120603,119472
unique,NaN,3,4192,4279,73,13118,4195
top,NaN,Transfer,2013-03-14 00:00:00,2013-12-28 00:00:00,GMD,296.90,2010-12-23 00:00:00
freq,NaN,61636,111,69,13062,2394,111
first,NaN,NaN,2004-01-28 00:00:00,2004-01-28 00:00:00,NaN,NaN,2004-02-11 00:00:00
last,NaN,NaN,2015-11-26 00:00:00,2015-11-26 00:00:00,NaN,NaN,2015-11-23 00:00:00
mean,5.366861e+07,NaN,NaN,NaN,NaN,NaN,NaN
std,4.628206e+05,NaN,NaN,NaN,NaN,NaN,NaN
min,5.013526e+07,NaN,NaN,NaN,NaN,NaN,NaN
25%,5.372980e+07,NaN,NaN,NaN,NaN,NaN,NaN


In [184]:
adt[adt.Admission_date.isnull() & adt.DISCHARGE_DATE.isnull()]

,RUID,Event,Admission_date,Event_Date,SRV_CODE,CHIEF_COMPLAINT,DISCHARGE_DATE
93,50135361,Transfer,NaT,2009-05-24,GMD,DEHYDRATION W/WEAKNESS,NaT
94,50135361,Transfer,NaT,2009-07-11,GMD,PNEUMONIA,NaT
95,50135361,Transfer,NaT,2010-04-29,GMD,PYELONEPHRITIS,NaT
96,50135361,Transfer,NaT,2010-10-03,GER,FALL,NaT
97,50135361,Transfer,NaT,2011-10-17,CAR,CHEST PAIN; ELEVATED TROPONIN,NaT
151,50135369,Transfer,NaT,2009-07-07,GMD,CHEST AND ABDOMINAL PAIN,NaT
152,50135369,Transfer,NaT,2010-03-30,PUL,COPD ACUTE HIP PAIN,NaT
153,50135369,Transfer,NaT,2010-12-21,CAR,UNSTABLE ANGINA,NaT
410,50135437,Transfer,NaT,2010-06-11,GMD,WOUND INFECTION,NaT
466,50135624,Transfer,NaT,2008-02-08,CAR,HEART TRANSPLANT,NaT


In [83]:
adt[adt.Admission_date.isnull() & ].Event.value_counts()

Transfer     1559
Discharge       2
Name: Event, dtype: int64

In [84]:
adt[adt.DISCHARGE_DATE.isnull()].Event.value_counts()

Transfer    1860
Admit        198
Name: Event, dtype: int64

In [85]:
adt[(adt.Admission_date.isnull()) & (adt.Event == 'Discharge')]

,RUID,Event,Admission_date,Event_Date,SRV_CODE,CHIEF_COMPLAINT,DISCHARGE_DATE
76409,53733158,Discharge,NaT,2007-08-06,TRA,STAT,2007-08-06
76575,53733172,Discharge,NaT,2013-02-17,PED,SEPSIS,2013-02-17


In [86]:
adt[76400:76420]

,RUID,Event,Admission_date,Event_Date,SRV_CODE,CHIEF_COMPLAINT,DISCHARGE_DATE
76400,53733157,Transfer,2009-10-25,2009-10-25,URO,UNSPEC DISORDER OF KIDNEY AND URETER,2009-10-29
76401,53733157,Discharge,2009-10-25,2009-10-29,URO,UNSPEC DISORDER OF KIDNEY AND URETER,2009-10-29
76402,53733157,Transfer,2011-11-30,2011-11-30,ONC,HYPOXIA;RENAL FAIL CANCER;DYSPNEA,2011-12-02
76403,53733157,Admit,2011-11-30,2011-11-30,ONC,HYPOXIA;RENAL FAIL CANCER;DYSPNEA,2011-12-02
76404,53733157,Discharge,2011-11-30,2011-12-02,ONC,HYPOXIA;RENAL FAIL CANCER;DYSPNEA,2011-12-02
76405,53733157,Admit,2009-12-19,2009-12-19,ONC,RENAL CELL CARINOMA,2009-12-25
76406,53733157,Transfer,2009-12-19,2009-12-19,ONC,RENAL CELL CARINOMA,2009-12-25
76407,53733157,Discharge,2009-12-19,2009-12-25,ONC,RENAL CELL CARINOMA,2009-12-25
76408,53733158,Transfer,NaT,2007-08-05,TRA,STAT,2007-08-06
76409,53733158,Discharge,NaT,2007-08-06,TRA,STAT,2007-08-06


In [87]:
adt[(adt.DISCHARGE_DATE.isnull()) & (adt.Event == 'Admit')]

,RUID,Event,Admission_date,Event_Date,SRV_CODE,CHIEF_COMPLAINT,DISCHARGE_DATE
1758,50141473,Admit,2015-06-26,2015-06-26,GMD,DETOX,NaT
1915,50143540,Admit,2011-01-06,2011-01-06,PED,RSV BRONCHIOLITIS; AV CANAL,NaT
2001,51319626,Admit,2013-11-19,2013-11-19,ORT,716.17/996.78,NaT
2258,52765702,Admit,2014-12-05,2014-12-05,GMD,BACK PAIN,NaT
2524,53719335,Admit,2014-09-15,2014-09-15,EMR,INTRACTABLE VOMITING AND DEHYDRATION,NaT
2841,53725969,Admit,2011-09-27,2011-09-27,EMR,SOB,NaT
2960,53727824,Admit,2013-11-13,2013-11-13,VAS,453.40 ACUTE VENOUS EMBOLISM AND THROMBOSIS,NaT
3637,53727886,Admit,2014-12-10,2014-12-10,OTO,POST OP T AND A BLEED,NaT
4023,53727917,Admit,2012-06-17,2012-06-17,ORT,813.23 RADIUS WITH ULNA; FRACTURE OF AND ULNA ...,NaT
5532,53728030,Admit,2015-08-12,2015-08-12,EMR,AIDS ENTEROPATHY; FAILURE TO THRIVE; DEHYDRATION,NaT


In [ ]:
# left join the phenotype to get DOB, filter out < 18 | PEDS SVC to create dataset w/o peds bc re-admission standards don't include peds

In [ ]:
# filter peds first, then make event categorical & sort everything so we have admit-transfer(s)-discharge
# you can simply grab all admit events for an individual once admit/discharge dates are squared away,
# and calculate days to next admit by subtracting discharge date at t-1 from admit date at t

In [197]:
adt_age = pd.merge(adt,pheno)
# admits = adt_age.Admission_date.dt
events = adt_age.Event_Date.dt
birthdays = adt_age.DOB.dt

adt_age['age'] = events.year - birthdays.year + ((events.month < birthdays.month) & (events.day < birthdays.day))
# above from https://stackoverflow.com/questions/2217488/age-from-birthdate-in-python/9754466#9754466

In [228]:
# getting rid of peds & psychiatric patients...
# we're removing these because they aren't part of the CMS criteria so 30-day readmits for them don't lose the hospital money
ped_svc = '|'.join(svc.SVC[svc.Desc.str.contains("CHILD|PED")])
psych_svc = '|'.join(svc.SVC[svc.Desc.str.contains("PSYCH")])

ped_filter = ((adt_age.age < 18) | (adt_age.SRV_CODE.str.contains(ped_svc)) & ~((adt_age.age > 35) & (adt_age.SRV_CODE.str.contains(ped_svc))))
# the ~ condition here contains a handful of rows that I think are coding errors -- very old patients admitted to pediatric services
# the cutoff is 35 because some pediatric cancer/cardiac/etc patients will continue with pediatric services for their original condition into adulthood

psych_filter = (adt_age.SRV_CODE.str.contains(psych_svc))

In [192]:
adt_age.head()

,RUID,Event,Admission_date,Event_Date,SRV_CODE,CHIEF_COMPLAINT,DISCHARGE_DATE,Sex,DOB,DOD,Race,age
0,50135262,Admit,2007-02-08,2007-02-08,ORT,R FEMUR FX,2007-02-12,F,1949-09-20,NaT,W,59.0
1,50135262,Transfer,2007-02-08,2007-02-08,ORT,R FEMUR FX,2007-02-12,F,1949-09-20,NaT,W,59.0
2,50135262,Transfer,2007-02-08,2007-02-09,ORT,R FEMUR FX,2007-02-12,F,1949-09-20,NaT,W,59.0
3,50135262,Discharge,2007-02-08,2007-02-12,ORT,R FEMUR FX,2007-02-12,F,1949-09-20,NaT,W,59.0
4,50135262,Admit,2007-08-03,2007-08-03,CAR,CP,2007-08-06,F,1949-09-20,NaT,W,59.0


In [187]:
adt_age.tail()

,RUID,Event,Admission_date,Event_Date,SRV_CODE,CHIEF_COMPLAINT,DISCHARGE_DATE,Sex,DOB,DOD,Race,age
121525,53736422,Transfer,2006-04-21,2006-04-30,TRA,STAT,2006-05-05,M,1948-05-28,NaT,H,59.0
121526,53736422,Transfer,2006-04-21,2006-05-02,TRA,STAT,2006-05-05,M,1948-05-28,NaT,H,59.0
121527,53736422,Discharge,2006-04-21,2006-05-05,TRA,STAT,2006-05-05,M,1948-05-28,NaT,H,59.0
121528,53736423,Admit,2006-04-23,2006-04-23,NUR,NaN,2006-04-25,M,2006-04-23,NaT,A,0.0
121529,53736423,Discharge,2006-04-23,2006-04-25,NUR,NaN,2006-04-25,M,2006-04-23,NaT,A,0.0


In [230]:
adt_cms = adt_age[~(ped_filter | psych_filter)]

In [153]:
cpt.head()

,RUID,CPT_Code,Event_date
0,50135262,00001,2004-11-28
1,50135262,00005,2012-05-03
2,50135262,00025,2007-02-08
3,50135262,00025,2004-11-16
4,50135262,00030,2007-02-08


In [165]:
hosp_ed_cpts = ["99217", "99218", "99219", "99220", "99221", "99222", "99223", "99224", "99225", "99226", "99231", "99232", "99233", "99234", "99235", "99236", "99238", "99239", "99251", "99252", "99253", "99254", "99255", "99289","99290", "99291", "99292", "99293", "99294", "99295","99296", "99297", "99356", "99357", "99358", "99359", "99433", "99435", "99460", "99461", "99462", "99463", "99466", "99467", "99468", "99469","99471", "99472", "99475", "99476", "99477", "99478", "99479", "99480", "99485", "99486", "99281", "99282", "99283", "99284", "99285"]
cpt_pat = "|".join(hosp_ed_cpts)

In [171]:
cpt_hosp = cpt[cpt.CPT_Code.str.match(cpt_pat)].sort_values(by=['RUID','Event_date','CPT_Code'])

In [215]:
adt_age[adt_age.RUID == 53728183]

,RUID,Event,Admission_date,Event_Date,SRV_CODE,CHIEF_COMPLAINT,DISCHARGE_DATE,Sex,DOB,DOD,Race,age
7684,53728183,Admit,2010-02-26,2010-02-26,PSY,296.24,2010-02-27,M,1959-06-13,NaT,B,51.0
7685,53728183,Transfer,2010-02-26,2010-02-26,PSY,296.24,2010-02-27,M,1959-06-13,NaT,B,51.0
7686,53728183,Transfer,2010-02-26,2010-02-27,PSY,296.24,2010-02-27,M,1959-06-13,NaT,B,51.0
7687,53728183,Discharge,2010-02-26,2010-02-27,PSY,296.24,2010-02-27,M,1959-06-13,NaT,B,51.0
7688,53728183,Admit,2010-02-27,2010-02-27,CAR,NSTEMI,2010-03-01,M,1959-06-13,NaT,B,51.0
7689,53728183,Transfer,2010-02-27,2010-02-27,PSY,NSTEMI,2010-03-01,M,1959-06-13,NaT,B,51.0
7690,53728183,Transfer,2010-02-27,2010-02-28,PSY,NSTEMI,2010-03-01,M,1959-06-13,NaT,B,51.0
7691,53728183,Transfer,2010-02-27,2010-03-01,CAR,NSTEMI,2010-03-01,M,1959-06-13,NaT,B,52.0
7692,53728183,Discharge,2010-02-27,2010-03-01,CAR,NSTEMI,2010-03-01,M,1959-06-13,NaT,B,52.0
7693,53728183,Admit,2010-03-01,2010-03-01,PSY,296.24,2010-03-04,M,1959-06-13,NaT,B,52.0


In [190]:
cpt_hosp[(cpt_hosp.RUID == 53736410)].sort_values(by=['Event_date','CPT_Code'])

,RUID,CPT_Code,Event_date
3818971,53736410,99223,2005-10-09
3818982,53736410,99231,2005-10-10
3818983,53736410,99231,2005-10-11
3818984,53736410,99231,2005-10-12
3818985,53736410,99231,2005-10-13
3818986,53736410,99231,2005-10-14
3818987,53736410,99231,2005-10-15
3819002,53736410,99238,2005-10-16
3818970,53736410,99223,2007-07-29
3818978,53736410,99231,2007-07-30


In [179]:
adt[adt.DISCHARGE_DATE.isnull()]

,RUID,Event,Admission_date,Event_Date,SRV_CODE,CHIEF_COMPLAINT,DISCHARGE_DATE
39,50135361,Transfer,NaT,2009-05-24,GMD,DEHYDRATION W/WEAKNESS,NaT
40,50135361,Transfer,NaT,2009-07-11,GMD,PNEUMONIA,NaT
38,50135361,Transfer,NaT,2010-04-29,GMD,PYELONEPHRITIS,NaT
41,50135361,Transfer,NaT,2010-10-03,GER,FALL,NaT
42,50135361,Transfer,NaT,2011-10-17,CAR,CHEST PAIN; ELEVATED TROPONIN,NaT
99,50135369,Transfer,NaT,2009-07-07,GMD,CHEST AND ABDOMINAL PAIN,NaT
98,50135369,Transfer,NaT,2010-03-30,PUL,COPD ACUTE HIP PAIN,NaT
100,50135369,Transfer,NaT,2010-12-21,CAR,UNSTABLE ANGINA,NaT
224,50135437,Transfer,NaT,2010-06-11,GMD,WOUND INFECTION,NaT
411,50135624,Transfer,NaT,2008-02-08,CAR,HEART TRANSPLANT,NaT


In [176]:
cpt_hosp[cpt_hosp.RUID == 53736417].sort_values(by=['Event_date','CPT_Code'])

,RUID,CPT_Code,Event_date
3823483,53736417,99282,2000-11-14
3823484,53736417,99283,2002-10-13
3823491,53736417,99284,2002-10-13
3823485,53736417,99283,2002-12-09
3823492,53736417,99284,2002-12-09
3823486,53736417,99284,2004-02-01
3823487,53736417,99284,2004-02-02
3823468,53736417,99223,2014-01-27
3823474,53736417,99233,2014-01-28
3823467,53736417,99222,2014-01-29


In [180]:
pd.date_range(start = "2014-04-26", end = "2014-10-30")

DatetimeIndex(['2014-04-26', '2014-04-27', '2014-04-28', '2014-04-29',
               '2014-04-30', '2014-05-01', '2014-05-02', '2014-05-03',
               '2014-05-04', '2014-05-05',
               ...
               '2014-10-21', '2014-10-22', '2014-10-23', '2014-10-24',
               '2014-10-25', '2014-10-26', '2014-10-27', '2014-10-28',
               '2014-10-29', '2014-10-30'],
              dtype='datetime64[ns]', length=188, freq='D')

In [ ]:
ex_cpt_codes = cpt_hosp[(cpt_hosp.RUID == 53736417) & (cpt_hosp.Event_date > "2014-04-14")]
ex_cpt_codes[ex_cpt_codes.CPT_Code.str.match("aqz")].shape

In [ ]:
# adt_cms_admits[didx].Readmit_time = [np.inf]*sum(didx)

# readmit_time = [(x - y)/datetime.timedelta(days=1) for x,y in zip(adt_cms_admits.Admission_date[1:],adt_cms_admits.DISCHARGE_DATE[:-1])]
# readmit_time.insert(0,np.inf)

# adt_cms_admits['Time_from_previous_discharge'] = readmit_time

# there's probably a way better and more pythonic way to do this :/

# adt_cms_admits['Time_to_readmit'] = 

# adt_cms_admits.Time_to_readmit[same_pt] = readmit_time

# for idx, row in adt_cms_admits.iterrows():
#     if (idx == 0):
#         pass
    
#     else:
#         row_above = adt_cms_admits.iloc[(idx-1)]
#         if row.RUID == row_above.RUID:
#             adt_cms_admits.Time_to_readmit.iloc[idx] = (row.Admission_date - row_above.DISCHARGE_DATE)/datetime.timedelta(days=1)
#         else:
#             passif ex_cpt_codes[ex_cpt_codes.CPT_Code.str.match(cpt_pat)].shape[0]:
    print("True")
else:
    print("False")

In [ ]:
adt_cms[(adt_cms.RUID == 50141473)]

In [ ]:
cpt_hosp[(cpt_hosp.RUID == 50141473) & (cpt_hosp.Event_date > "2015-06-25")]

In [ ]:
adt_cms[(adt_cms.RUID == 53736417) & (adt_cms.Admission_date == "2014-04-14")]